In [80]:

#Do not run all cells at once
import os
import time

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [81]:
os.chdir("/Users/snehabhatt/Documents")
# /Users/snehabhatt/Documents

In [82]:
#Information we want to scrape
MainData = pd.DataFrame(columns = ['Summary','Date', 'OverallRating', 'WorkLifeBalance', 'CultureValues', 'CareerOpportunities', 'CompBenefits', 'SeniorManagement', 'Pros', 'Cons'])

In [105]:
#Define a function that is reusable. We need to scrape many pages of many companies
#Define a function that is reusable. We need to scrape many pages of many companies
def SCRAPER(url):
    global MainData
    global soup
    #Define some lists
    WorkLifeBalance = []
    CultureValues=[]
    CareerOpportunities=[]
    CompBenefits=[]
    SeniorManagement=[]    
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(url,headers=hdr)
    page = urlopen(req)
    soup = BeautifulSoup(page, "html.parser")
    #Get the summary
    Summary = soup.find_all('span', attrs = {'class':'summary'})
    SummaryClean = []
    for x in Summary:
        SummaryClean.append(x.text)
    
    #Get Postdate
    PostDate = soup.find_all('time', attrs = {'class':'date subtle small'})
    PostDateClean = []
    for x in PostDate:
        PostDateClean.append(x['datetime'])
        
    #Get overall rating
    ValueTitle = soup.find_all('span', attrs = {'class':'value-title'})
    ValueTitleClean = []
    for x in ValueTitle:
        ValueTitleClean.append(x['title'])
    
    #Get subratings. Since a user can choose not to rate in some categories, things are more complicated
    SubRatings= soup.find_all('div', attrs = {'class':'subRatings module'})
    for subrating in SubRatings:
        RatingCat=subrating.find_all('div', attrs = {'class':'minor'})
        RatingCatClean = []
        for x in RatingCat:
            RatingCatClean.append(x.text)
        RatingVal=subrating.find_all('span', attrs = {'class':'gdBars gdRatings med '})
        RatingValClean = []
        for x in RatingVal:
            RatingValClean.append(x['title'])
        try:
            index = RatingCatClean.index('Work/Life Balance')
        except ValueError:
            index=100
        RatingValue = WorkLifeBalance.append(None) if index==100 else WorkLifeBalance.append(RatingValClean[index])
        try:
            index = RatingCatClean.index('Culture & Values')
        except ValueError:
            index=100
        RatingValue = CultureValues.append(None) if index==100 else CultureValues.append(RatingValClean[index])
        try:
            index = RatingCatClean.index('Career Opportunities')
        except ValueError:
            index=100
        RatingValue = CareerOpportunities.append(None) if index==100 else CareerOpportunities.append(RatingValClean[index])
        try:
            index = RatingCatClean.index('Comp & Benefits')
        except ValueError:
            index=100
        RatingValue = CompBenefits.append(None) if index==100 else CompBenefits.append(RatingValClean[index])
        try:
            index = RatingCatClean.index('Senior Management')
        except ValueError:
            index=100
        RatingValue = SeniorManagement.append(None) if index==100 else SeniorManagement.append(RatingValClean[index]) 
        
    #Get Pros
    Pros = soup.find_all('p', attrs = {'class':' pros mainText truncateThis wrapToggleStr'})
    ProsClean = []
    for x in Pros:
        ProsClean.append(x.text)    
    
    #Get Cons
    Cons = soup.find_all('p', attrs = {'class':' cons mainText truncateThis wrapToggleStr'})
    ConsClean = []
    for x in Cons:
        ConsClean.append(x.text)    
        #Get the overall rating
    DATA = pd.DataFrame() 
    MainData=MainData.append(DATA,)
    DATA.loc[:,'Summary'] = pd.Series(SummaryClean)
    DATA.loc[:,'Date'] = pd.Series(PostDateClean)
    DATA.loc[:,'OverallRating'] = pd.Series(ValueTitleClean)
    DATA.loc[:,'WorkLifeBalance'] = pd.Series(WorkLifeBalance)
    DATA.loc[:,'CultureValues'] = pd.Series(CultureValues)
    DATA.loc[:,'CareerOpportunities'] = pd.Series(CareerOpportunities)
    DATA.loc[:,'CompBenefits'] = pd.Series(CompBenefits)
    DATA.loc[:,'SeniorManagement'] = pd.Series(SeniorManagement)
    DATA.loc[:,'Pros'] = pd.Series(ProsClean)
    DATA.loc[:,'Cons'] = pd.Series(ConsClean)
    MainData=MainData.append(DATA,)
    #Wait a little
time.sleep(30)
    

In [106]:
#Scrape Wonderful Glassdoor Data
for x in range(1,2):
    #url="https://www.glassdoor.com/Reviews/The-Wonderful-Company-Reviews-E1005987_P"+str(x)+".htm?sort.sortType=RD&sort.ascending=false"
    url="https://www.glassdoor.co.in/Reviews/Amazon-Reviews-E6036_P"+str(x)+".htm"
    # https://www.glassdoor.co.in/Reviews/Amazon-Reviews-E6036_P3.htm
    print(url)
    SCRAPER(url)

https://www.glassdoor.co.in/Reviews/Amazon-Reviews-E6036_P1.htm


In [107]:
MainData.head()

,Summary,Date,OverallRating,WorkLifeBalance,CultureValues,CareerOpportunities,CompBenefits,SeniorManagement,Pros,Cons
0,0,Thu Jul 18 2019 11:54:48 GMT-0700 (PDT),3.4,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,0,Tue Jul 16 2019 08:39:23 GMT-0700 (PDT),5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,0,Thu Jul 11 2019 08:25:13 GMT-0700 (PDT),4.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,0,Thu Jul 04 2019 09:33:37 GMT-0700 (PDT),5.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,0,Sun Jun 30 2019 23:38:45 GMT-0700 (PDT),3.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [104]:
#write files
MainData=MainData.reset_index(drop=True)
MainData.to_csv('Glassdoor_Data.csv', mode = 'w')